In [1]:
import os, pathlib

file_path = os.path.dirname(os.path.realpath("__file__"))
home_dir = pathlib.Path(file_path).parent.parent
os.chdir(home_dir)

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from src.common import load_layer_activations
from src.mnist.data import get_mnist_train_test
from src.mnist.perfectAE.file_names import SAVED_CLASSIFIER_PATH, SAVED_CLADEC_PATH, SAVED_REFAE_PATH

tf.random.set_seed(1234)
np.random.seed(1234)

In [3]:
(dense_train_acts, dense_test_acts), (conv_train_acts, conv_test_acts) = load_layer_activations(SAVED_CLASSIFIER_PATH)
train_images, test_images = get_mnist_train_test()

In [4]:
# load RefAE model
refae_dense_path = SAVED_REFAE_PATH.joinpath('dense')
refae_conv_path = SAVED_REFAE_PATH.joinpath('conv')
refae_dense = keras.models.load_model(refae_dense_path)
refae_conv = keras.models.load_model(refae_conv_path)

refae_dense.evaluate(test_images, test_images)

2022-06-11 10:13:05.553256: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-11 10:13:05.669043: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-11 10:13:05.669918: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-11 10:13:05.673162: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

313/313 [==============================] - 6s 5ms/step - loss: 0.0043


0.004286200739443302

In [5]:
alphas = [0, 1, 50, 99]

for alpha in alphas:
    cladec_path = SAVED_CLADEC_PATH.joinpath(f'{alpha:2}')
    cladec_dense_path = cladec_path.joinpath("dense")
    cladec_dense = keras.models.load_model(cladec_dense_path)
    cladec_dense.compile(optimizer='Adam', loss='mean_squared_error')
    print(f'Evaluating {cladec_dense_path} on test with alpha={alpha}')
    cladec_dense.evaluate(dense_test_acts, test_images)
    

Evaluating /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/perfectAE/cladec/ 0/dense on test with alpha=0
313/313 [==============================] - 1s 2ms/step - loss: 0.0235
Evaluating /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/perfectAE/cladec/ 1/dense on test with alpha=1
313/313 [==============================] - 1s 2ms/step - loss: 0.0368
Evaluating /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/perfectAE/cladec/50/dense on test with alpha=50
313/313 [==============================] - 1s 2ms/step - loss: 0.0725
Evaluating /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/perfectAE/cladec/99/dense on test with alpha=99
313/313 [==============================] - 1s 2ms/step - loss: 0.1631
